In [138]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_circles
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import MinMaxScaler

In [139]:
scaler = MinMaxScaler()

In [140]:
class KernelSVM:
    def __init__(self, C, degree, gamma, kernel):
        self.C = C
        self.gamma = gamma
        self.kernel = kernel
        self.degree = degree
    
    def rbf(self, X1, X2):
        return np.exp(-self.gamma * np.sum((X2 - X1[:,np.newaxis])**2, axis = -1))
    
    def poly(self, X1, X2, k=0.5):
        return (np.matmul(X1, np.transpose(X2))+k)**self.degree
    
    def linear(self, X):
        return X
    
    def loss(self, y, y_cap):
        hinge_loss_params = 1-y*y_cap
        hinge_loss = np.sum(np.where(hinge_loss_params<0, 0, hinge_loss_params)) / len(y)
        regularization_loss = sum(self.W)
        return self.C * hinge_loss + regularization_loss
    
    def compute_gradients(self, X, Y, Y_cap, n):
        # differentiation of [1/2 * W**2 + C * (1-y*(W*X+b))]
        
        dW = (-self.C * np.multiply(Y, X.T).T).T + 1
        for i in range(self.W.shape[0]):
            dW[i] = np.where(1-Y_cap <= 0, self.W[i], dW[i])
        dW = np.sum(dW, axis=1)
        dW = dW/n
        
        db = -Y * self.b
        db = np.where(1-Y_cap <= 0, 0, db)
        db = sum(db)
        db = db / n
        
        return dW, db
    
    def train(self, X, Y, epochs, lr):
        
        self.x_start = X.copy()
        
        if self.kernel == "rbf":
            X = self.rbf(X, X)
        elif self.kernel == "poly":
            X = self.poly(X, X)
        elif self.kernel == "linear":
            X = self.linear(X)
        else:
            print("Unexpected kernel type...")
        
        n_instances = X.shape[0]
        n_features = X.shape[1]
        
        self.W = np.zeros(n_features)
        self.b = 0.0
        
        for epoch in range(epochs):
            y_cap = np.dot(X,self.W) + self.b
            loss = self.loss(Y, y_cap)
            print("Epoch No: " + str(epoch) + ", Loss: " + str(loss))
            dW, db = self.compute_gradients(X, Y, y_cap, n_instances)
            self.W -= lr * dW
            self.b -= lr * db
    
    def predict(self, X):
        if self.kernel == "rbf":
            X = self.rbf(X, self.x_start)
        elif self.kernel == "poly":
            X = self.poly(X, self.x_start)
        elif self.kernel == "linear":
            X = self.linear(X)
        return np.where(np.dot(X, self.W) + self.b > 0, 1, -1)
    
    def evaluate(self, X, Y):
        y_cap = self.predict(X)
        acc = len(np.where(y_cap == Y)[0]) / len(Y)
        return acc
    
    def get_weights(self):
        return self.W

In [141]:
def decision_boundary(model, X, Y):
    X1_min, X1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    X2_min, X2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    
    XX, YY = np.meshgrid(np.arange(X1_min, X1_max, 0.02), np.arange(X2_min, X2_max, 0.02))

    Z = model.predict(np.c_[XX.ravel(), YY.ravel()])
    Z = Z.reshape(XX.shape)
    
    plt.scatter(X[np.where(Y==-1)[0], 0], X[np.where(Y==-1)[0], 1], label='-1')
    plt.scatter(X[np.where(Y==1)[0], 0], X[np.where(Y==1)[0], 1], label='1')
    plt.contour(XX, YY, Z, cmap=plt.cm.Paired)
    plt.legend()
    plt.show()

In [142]:
# Q. 3.4

In [143]:
X, Y = load_breast_cancer(return_X_y=True)
Y[np.where(Y==0)] = -1

In [144]:
scaler.fit(X)
X = scaler.transform(X)

In [145]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=95, random_state=1)

In [146]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=95, random_state=1)

In [147]:
# RBF kernel
# C=1, gamma = 1
svm = KernelSVM(C=1, degree=None, gamma=1, kernel="rbf")
svm.train(X_train, Y_train, epochs=1000, lr=0.01)

Epoch No: 0, Loss: 1.0
Epoch No: 1, Loss: 1.4514997690974307
Epoch No: 2, Loss: 1.9012847330157177
Epoch No: 3, Loss: 2.332563941622887
Epoch No: 4, Loss: 2.0212600711749307
Epoch No: 5, Loss: 1.7419968104344807
Epoch No: 6, Loss: 1.6823134909766049
Epoch No: 7, Loss: 1.5749939608736072
Epoch No: 8, Loss: 1.494137014651634
Epoch No: 9, Loss: 1.4046666537082997
Epoch No: 10, Loss: 1.3039661424787288
Epoch No: 11, Loss: 1.2272226188560689
Epoch No: 12, Loss: 1.1369088780837848
Epoch No: 13, Loss: 1.0591395332468232
Epoch No: 14, Loss: 0.9724247474736829
Epoch No: 15, Loss: 0.8831645248860687
Epoch No: 16, Loss: 0.8117715026294137
Epoch No: 17, Loss: 0.7221606465740658
Epoch No: 18, Loss: 0.6494560069540501
Epoch No: 19, Loss: 0.5686820920550286
Epoch No: 20, Loss: 0.5056905268363971
Epoch No: 21, Loss: 0.42266735095986796
Epoch No: 22, Loss: 0.35164686305160514
Epoch No: 23, Loss: 0.28273309295234145
Epoch No: 24, Loss: 0.21557669274825936
Epoch No: 25, Loss: 0.15743907533390789
Epoch No

Epoch No: 230, Loss: -4.853497371244821
Epoch No: 231, Loss: -4.861439893230338
Epoch No: 232, Loss: -4.86930220601601
Epoch No: 233, Loss: -4.872656230632159
Epoch No: 234, Loss: -4.8804071640238025
Epoch No: 235, Loss: -4.888080588081525
Epoch No: 236, Loss: -4.895677277898676
Epoch No: 237, Loss: -4.903198000817651
Epoch No: 238, Loss: -4.910643211745478
Epoch No: 239, Loss: -4.91358335187849
Epoch No: 240, Loss: -4.920925014057667
Epoch No: 241, Loss: -4.9281932596150515
Epoch No: 242, Loss: -4.9353888227168605
Epoch No: 243, Loss: -4.942512430187655
Epoch No: 244, Loss: -4.94956480158374
Epoch No: 245, Loss: -4.9565454824019035
Epoch No: 246, Loss: -4.959027453309338
Epoch No: 247, Loss: -4.965914674474202
Epoch No: 248, Loss: -4.972733023427425
Epoch No: 249, Loss: -4.97948318889111
Epoch No: 250, Loss: -4.986165451331478
Epoch No: 251, Loss: -4.9927776524447545
Epoch No: 252, Loss: -4.994895364393827
Epoch No: 253, Loss: -5.001420931616093
Epoch No: 254, Loss: -5.007881243166143

Epoch No: 485, Loss: -5.522511089671334
Epoch No: 486, Loss: -5.523753803121439
Epoch No: 487, Loss: -5.5205624784438365
Epoch No: 488, Loss: -5.521825085080438
Epoch No: 489, Loss: -5.5230750656506755
Epoch No: 490, Loss: -5.524312546415221
Epoch No: 491, Loss: -5.5255376523721065
Epoch No: 492, Loss: -5.526750507269429
Epoch No: 493, Loss: -5.527949971392099
Epoch No: 494, Loss: -5.5247175316317305
Epoch No: 495, Loss: -5.525938587736453
Epoch No: 496, Loss: -5.527147433280134
Epoch No: 497, Loss: -5.528344190368381
Epoch No: 498, Loss: -5.529528979885737
Epoch No: 499, Loss: -5.5307011664871455
Epoch No: 500, Loss: -5.527440712642971
Epoch No: 501, Loss: -5.5286345369375915
Epoch No: 502, Loss: -5.5298164229892555
Epoch No: 503, Loss: -5.530986490180407
Epoch No: 504, Loss: -5.532144856699656
Epoch No: 505, Loss: -5.533291362056259
Epoch No: 506, Loss: -5.53000453350829
Epoch No: 507, Loss: -5.531172719594251
Epoch No: 508, Loss: -5.532329223819346
Epoch No: 509, Loss: -5.5334741630

Epoch No: 740, Loss: -5.574497576552321
Epoch No: 741, Loss: -5.570798419050799
Epoch No: 742, Loss: -5.5715586662813354
Epoch No: 743, Loss: -5.57231131103957
Epoch No: 744, Loss: -5.573056429350222
Epoch No: 745, Loss: -5.573794096477763
Epoch No: 746, Loss: -5.574524386934024
Epoch No: 747, Loss: -5.575246484229065
Epoch No: 748, Loss: -5.571540711090995
Epoch No: 749, Loss: -5.572293535401137
Epoch No: 750, Loss: -5.573038831468176
Epoch No: 751, Loss: -5.57377667457453
Epoch No: 752, Loss: -5.574507139249834
Epoch No: 753, Loss: -5.575229894459042
Epoch No: 754, Loss: -5.571523806635724
Epoch No: 755, Loss: -5.572276799990403
Epoch No: 756, Loss: -5.573022263411546
Epoch No: 757, Loss: -5.573760272198475
Epoch No: 758, Loss: -5.574490900897537
Epoch No: 759, Loss: -5.575214223309607
Epoch No: 760, Loss: -5.575929052373576
Epoch No: 761, Loss: -5.5722168197227
Epoch No: 762, Loss: -5.57296288294652
Epoch No: 763, Loss: -5.573701485538098
Epoch No: 764, Loss: -5.574432702103759
Epoc

Epoch No: 983, Loss: -5.574959636035711
Epoch No: 984, Loss: -5.575678271096391
Epoch No: 985, Loss: -5.576389719806477
Epoch No: 986, Loss: -5.577094054029458
Epoch No: 987, Loss: -5.577791344910213
Epoch No: 988, Loss: -5.578481582229422
Epoch No: 989, Loss: -5.574742656603453
Epoch No: 990, Loss: -5.575463461458465
Epoch No: 991, Loss: -5.576177058264922
Epoch No: 992, Loss: -5.57688351910332
Epoch No: 993, Loss: -5.577582915333333
Epoch No: 994, Loss: -5.578275317601053
Epoch No: 995, Loss: -5.578959837866565
Epoch No: 996, Loss: -5.575216998237742
Epoch No: 997, Loss: -5.575933059676412
Epoch No: 998, Loss: -5.57664196050069
Epoch No: 999, Loss: -5.577343772316726


In [148]:
svm.evaluate(X_test, Y_test)

0.8526315789473684

In [149]:
svm.get_weights()

array([ 0.0181608 , -0.1496459 ,  0.02897885, -0.0014853 ,  0.03998996,
       -0.08756854,  0.04201006,  0.03099521, -0.12190369, -0.13464109,
        0.02694225, -0.00980595, -0.08515236,  0.04974306, -0.12229034,
        0.02969832, -0.1318829 , -0.06650556,  0.02745056, -0.11804303,
        0.04404746, -0.116645  ,  0.01906233, -0.09873728,  0.05053236,
       -0.10158956,  0.0322456 ,  0.03503312,  0.0280909 ,  0.02337525,
       -0.10077835, -0.01391094,  0.00924964,  0.05198913, -0.04076739,
       -0.08157782, -0.13306978,  0.04573612,  0.02446617,  0.02966365,
        0.00643222,  0.04585064, -0.04098442, -0.03391809, -0.10817671,
        0.0499687 , -0.07038874,  0.0481085 ,  0.03232727,  0.05085412,
       -0.06842382,  0.04196921, -0.03456094, -0.01668542,  0.0431763 ,
        0.04512983, -0.13420745,  0.03617097,  0.04423067,  0.03547642,
       -0.14250722, -0.01935492,  0.02815315, -0.09853931,  0.01956624,
        0.04313695,  0.04262188, -0.12944383, -0.10250585,  0.05

In [150]:
# Polynomial kernel
# C=1, degree = 20
svm = KernelSVM(C=1, degree=1, gamma=None, kernel="poly")
svm.train(X_train, Y_train, epochs=10000, lr=0.1)

Epoch No: 0, Loss: 1.0
Epoch No: 1, Loss: 3.5133276607066692
Epoch No: 2, Loss: 3.1997257653484006
Epoch No: 3, Loss: 2.917484059525963
Epoch No: 4, Loss: 2.6634665242857696
Epoch No: 5, Loss: 2.434895855491355
Epoch No: 6, Loss: 2.686475548821804
Epoch No: 7, Loss: 2.4555588646520263
Epoch No: 8, Loss: 2.2477338488992245
Epoch No: 9, Loss: 2.0610750722803903
Epoch No: 10, Loss: 3.0359201028534857
Epoch No: 11, Loss: 2.770058963280537
Epoch No: 12, Loss: 2.5307839376648844
Epoch No: 13, Loss: 2.3154364146107973
Epoch No: 14, Loss: 2.1216236438621188
Epoch No: 15, Loss: 1.9475014775308537
Epoch No: 16, Loss: 3.144946892256507
Epoch No: 17, Loss: 2.868183073743257
Epoch No: 18, Loss: 2.6190956370813323
Epoch No: 19, Loss: 2.394916944085601
Epoch No: 20, Loss: 2.1931561203894425
Epoch No: 21, Loss: 2.0115713790628993
Epoch No: 22, Loss: 1.8493626373756435
Epoch No: 23, Loss: 5.42038953123382
Epoch No: 24, Loss: 4.916081448822836
Epoch No: 25, Loss: 4.462204174652953
Epoch No: 26, Loss: 4.

Epoch No: 270, Loss: 3.420035386470856
Epoch No: 271, Loss: 3.115762718536172
Epoch No: 272, Loss: 2.841917317394957
Epoch No: 273, Loss: 2.5954564563678617
Epoch No: 274, Loss: 2.373641681443477
Epoch No: 275, Loss: 2.1740083840115294
Epoch No: 276, Loss: 1.9943384163227775
Epoch No: 277, Loss: 1.8343806060633971
Epoch No: 278, Loss: 6.102735761767522
Epoch No: 279, Loss: 5.5301930563031725
Epoch No: 280, Loss: 5.014904621385254
Epoch No: 281, Loss: 4.55114502995913
Epoch No: 282, Loss: 4.1337613976756185
Epoch No: 283, Loss: 3.7581161286204576
Epoch No: 284, Loss: 3.420035386470813
Epoch No: 285, Loss: 3.1157627185361325
Epoch No: 286, Loss: 2.84191731739492
Epoch No: 287, Loss: 2.5954564563678297
Epoch No: 288, Loss: 2.373641681443448
Epoch No: 289, Loss: 2.1740083840115036
Epoch No: 290, Loss: 1.9943384163227544
Epoch No: 291, Loss: 1.834380606063382
Epoch No: 292, Loss: 6.102735761767503
Epoch No: 293, Loss: 5.530193056303155
Epoch No: 294, Loss: 5.01490462138524
Epoch No: 295, Lo

Epoch No: 546, Loss: 5.014904621385236
Epoch No: 547, Loss: 4.5511450299591125
Epoch No: 548, Loss: 4.1337613976756025
Epoch No: 549, Loss: 3.7581161286204434
Epoch No: 550, Loss: 3.420035386470799
Epoch No: 551, Loss: 3.1157627185361223
Epoch No: 552, Loss: 2.841917317394909
Epoch No: 553, Loss: 2.595456456367819
Epoch No: 554, Loss: 2.3736416814434387
Epoch No: 555, Loss: 2.1740083840114957
Epoch No: 556, Loss: 1.9943384163227469
Epoch No: 557, Loss: 1.8343806060633776
Epoch No: 558, Loss: 6.102735761767498
Epoch No: 559, Loss: 5.530193056303146
Epoch No: 560, Loss: 5.014904621385236
Epoch No: 561, Loss: 4.5511450299591125
Epoch No: 562, Loss: 4.1337613976756025
Epoch No: 563, Loss: 3.758116128620443
Epoch No: 564, Loss: 3.420035386470799
Epoch No: 565, Loss: 3.1157627185361214
Epoch No: 566, Loss: 2.841917317394909
Epoch No: 567, Loss: 2.5954564563678186
Epoch No: 568, Loss: 2.3736416814434387
Epoch No: 569, Loss: 2.1740083840114957
Epoch No: 570, Loss: 1.9943384163227473
Epoch No: 

Epoch No: 825, Loss: 5.530193056303147
Epoch No: 826, Loss: 5.014904621385236
Epoch No: 827, Loss: 4.5511450299591125
Epoch No: 828, Loss: 4.1337613976756025
Epoch No: 829, Loss: 3.7581161286204434
Epoch No: 830, Loss: 3.420035386470799
Epoch No: 831, Loss: 3.115762718536122
Epoch No: 832, Loss: 2.841917317394909
Epoch No: 833, Loss: 2.595456456367819
Epoch No: 834, Loss: 2.373641681443439
Epoch No: 835, Loss: 2.1740083840114957
Epoch No: 836, Loss: 1.994338416322747
Epoch No: 837, Loss: 1.8343806060633776
Epoch No: 838, Loss: 6.102735761767498
Epoch No: 839, Loss: 5.530193056303146
Epoch No: 840, Loss: 5.014904621385236
Epoch No: 841, Loss: 4.5511450299591125
Epoch No: 842, Loss: 4.1337613976756025
Epoch No: 843, Loss: 3.758116128620443
Epoch No: 844, Loss: 3.420035386470799
Epoch No: 845, Loss: 3.1157627185361214
Epoch No: 846, Loss: 2.841917317394909
Epoch No: 847, Loss: 2.595456456367819
Epoch No: 848, Loss: 2.3736416814434387
Epoch No: 849, Loss: 2.1740083840114957
Epoch No: 850, 

Epoch No: 1096, Loss: 3.420035386470799
Epoch No: 1097, Loss: 3.1157627185361214
Epoch No: 1098, Loss: 2.841917317394909
Epoch No: 1099, Loss: 2.595456456367819
Epoch No: 1100, Loss: 2.373641681443439
Epoch No: 1101, Loss: 2.1740083840114957
Epoch No: 1102, Loss: 1.9943384163227473
Epoch No: 1103, Loss: 1.8343806060633776
Epoch No: 1104, Loss: 6.102735761767498
Epoch No: 1105, Loss: 5.530193056303148
Epoch No: 1106, Loss: 5.014904621385236
Epoch No: 1107, Loss: 4.5511450299591125
Epoch No: 1108, Loss: 4.1337613976756025
Epoch No: 1109, Loss: 3.7581161286204434
Epoch No: 1110, Loss: 3.420035386470799
Epoch No: 1111, Loss: 3.1157627185361214
Epoch No: 1112, Loss: 2.841917317394909
Epoch No: 1113, Loss: 2.595456456367819
Epoch No: 1114, Loss: 2.3736416814434387
Epoch No: 1115, Loss: 2.1740083840114957
Epoch No: 1116, Loss: 1.994338416322747
Epoch No: 1117, Loss: 1.8343806060633776
Epoch No: 1118, Loss: 6.102735761767498
Epoch No: 1119, Loss: 5.530193056303146
Epoch No: 1120, Loss: 5.01490

...

Epoch No: 8716, Loss: 2.3736416814434387
Epoch No: 8717, Loss: 2.1740083840114957
Epoch No: 8718, Loss: 1.994338416322747
Epoch No: 8719, Loss: 1.8343806060633776
Epoch No: 8720, Loss: 6.102735761767498
Epoch No: 8721, Loss: 5.530193056303147
Epoch No: 8722, Loss: 5.014904621385236
Epoch No: 8723, Loss: 4.5511450299591125
Epoch No: 8724, Loss: 4.1337613976756025
Epoch No: 8725, Loss: 3.7581161286204434
Epoch No: 8726, Loss: 3.420035386470799
Epoch No: 8727, Loss: 3.1157627185361214
Epoch No: 8728, Loss: 2.841917317394909
Epoch No: 8729, Loss: 2.595456456367819
Epoch No: 8730, Loss: 2.373641681443439
Epoch No: 8731, Loss: 2.1740083840114957
Epoch No: 8732, Loss: 1.9943384163227473
Epoch No: 8733, Loss: 1.8343806060633776
Epoch No: 8734, Loss: 6.102735761767498
Epoch No: 8735, Loss: 5.530193056303146
Epoch No: 8736, Loss: 5.014904621385236
Epoch No: 8737, Loss: 4.5511450299591125
Epoch No: 8738, Loss: 4.1337613976756025
Epoch No: 8739, Loss: 3.7581161286204425
Epoch No: 8740, Loss: 3.420

Epoch No: 9006, Loss: 3.420035386470799
Epoch No: 9007, Loss: 3.1157627185361214
Epoch No: 9008, Loss: 2.841917317394909
Epoch No: 9009, Loss: 2.595456456367819
Epoch No: 9010, Loss: 2.373641681443439
Epoch No: 9011, Loss: 2.1740083840114957
Epoch No: 9012, Loss: 1.9943384163227473
Epoch No: 9013, Loss: 1.8343806060633776
Epoch No: 9014, Loss: 6.102735761767498
Epoch No: 9015, Loss: 5.530193056303146
Epoch No: 9016, Loss: 5.014904621385236
Epoch No: 9017, Loss: 4.5511450299591125
Epoch No: 9018, Loss: 4.1337613976756025
Epoch No: 9019, Loss: 3.7581161286204425
Epoch No: 9020, Loss: 3.420035386470799
Epoch No: 9021, Loss: 3.115762718536122
Epoch No: 9022, Loss: 2.841917317394909
Epoch No: 9023, Loss: 2.595456456367819
Epoch No: 9024, Loss: 2.3736416814434387
Epoch No: 9025, Loss: 2.1740083840114957
Epoch No: 9026, Loss: 1.9943384163227473
Epoch No: 9027, Loss: 1.8343806060633776
Epoch No: 9028, Loss: 6.102735761767498
Epoch No: 9029, Loss: 5.530193056303146
Epoch No: 9030, Loss: 5.01490

Epoch No: 9290, Loss: 2.3736416814434387
Epoch No: 9291, Loss: 2.1740083840114957
Epoch No: 9292, Loss: 1.994338416322747
Epoch No: 9293, Loss: 1.8343806060633776
Epoch No: 9294, Loss: 6.102735761767498
Epoch No: 9295, Loss: 5.530193056303146
Epoch No: 9296, Loss: 5.014904621385236
Epoch No: 9297, Loss: 4.5511450299591125
Epoch No: 9298, Loss: 4.1337613976756025
Epoch No: 9299, Loss: 3.7581161286204425
Epoch No: 9300, Loss: 3.420035386470799
Epoch No: 9301, Loss: 3.1157627185361214
Epoch No: 9302, Loss: 2.841917317394909
Epoch No: 9303, Loss: 2.595456456367819
Epoch No: 9304, Loss: 2.3736416814434382
Epoch No: 9305, Loss: 2.1740083840114957
Epoch No: 9306, Loss: 1.9943384163227473
Epoch No: 9307, Loss: 1.8343806060633776
Epoch No: 9308, Loss: 6.102735761767498
Epoch No: 9309, Loss: 5.530193056303146
Epoch No: 9310, Loss: 5.014904621385236
Epoch No: 9311, Loss: 4.5511450299591125
Epoch No: 9312, Loss: 4.1337613976756025
Epoch No: 9313, Loss: 3.7581161286204434
Epoch No: 9314, Loss: 3.42

Epoch No: 9568, Loss: 2.841917317394909
Epoch No: 9569, Loss: 2.595456456367819
Epoch No: 9570, Loss: 2.373641681443439
Epoch No: 9571, Loss: 2.1740083840114957
Epoch No: 9572, Loss: 1.9943384163227473
Epoch No: 9573, Loss: 1.8343806060633776
Epoch No: 9574, Loss: 6.102735761767498
Epoch No: 9575, Loss: 5.530193056303146
Epoch No: 9576, Loss: 5.014904621385236
Epoch No: 9577, Loss: 4.5511450299591125
Epoch No: 9578, Loss: 4.1337613976756025
Epoch No: 9579, Loss: 3.7581161286204425
Epoch No: 9580, Loss: 3.420035386470799
Epoch No: 9581, Loss: 3.1157627185361214
Epoch No: 9582, Loss: 2.841917317394909
Epoch No: 9583, Loss: 2.595456456367819
Epoch No: 9584, Loss: 2.3736416814434387
Epoch No: 9585, Loss: 2.1740083840114957
Epoch No: 9586, Loss: 1.9943384163227473
Epoch No: 9587, Loss: 1.8343806060633776
Epoch No: 9588, Loss: 6.102735761767498
Epoch No: 9589, Loss: 5.530193056303147
Epoch No: 9590, Loss: 5.014904621385236
Epoch No: 9591, Loss: 4.5511450299591125
Epoch No: 9592, Loss: 4.1337

Epoch No: 9849, Loss: 2.595456456367819
Epoch No: 9850, Loss: 2.373641681443439
Epoch No: 9851, Loss: 2.1740083840114957
Epoch No: 9852, Loss: 1.9943384163227473
Epoch No: 9853, Loss: 1.8343806060633776
Epoch No: 9854, Loss: 6.102735761767498
Epoch No: 9855, Loss: 5.530193056303146
Epoch No: 9856, Loss: 5.014904621385236
Epoch No: 9857, Loss: 4.5511450299591125
Epoch No: 9858, Loss: 4.1337613976756025
Epoch No: 9859, Loss: 3.7581161286204425
Epoch No: 9860, Loss: 3.420035386470799
Epoch No: 9861, Loss: 3.115762718536122
Epoch No: 9862, Loss: 2.841917317394909
Epoch No: 9863, Loss: 2.595456456367819
Epoch No: 9864, Loss: 2.3736416814434387
Epoch No: 9865, Loss: 2.1740083840114957
Epoch No: 9866, Loss: 1.9943384163227473
Epoch No: 9867, Loss: 1.8343806060633776
Epoch No: 9868, Loss: 6.102735761767498
Epoch No: 9869, Loss: 5.530193056303146
Epoch No: 9870, Loss: 5.014904621385236
Epoch No: 9871, Loss: 4.5511450299591125
Epoch No: 9872, Loss: 4.1337613976756025
Epoch No: 9873, Loss: 3.7581

In [151]:
svm.evaluate(X_test, Y_test)

0.6